# Survey contract

***Before running this notebook, you might want to use Jupyter's "clear output" function to erase the results of the previous execution of this notebook. That will make more apparent what has been executed in the current session.***

This notebook is an excercise that will help you get started with Marlowe. The goal is to design a Survey contract using the [Marlowe Playground](https://play.marlowe.iohk.io/) and then execute it on the Cardano `preprod` network using the [Marlowe Runtime](https://docs.marlowe.iohk.io/docs/developer-tools/runtime/marlowe-runtime) 

TODO: add prerequesites section with link for
* Marlowe Language tutorial
* other?

## Preliminaries

If you haven't done it before, see the [preliminaries](../../docs/preliminaries.md) for information on setting up one's environment for using this notebook.

The first step is to check we have all the required tools and environment variables available to the notebook. 

In [ ]:
export SCRIPTS=$(realpath ../../scripts)
export KEYS=$(realpath ../../keys)
source $SCRIPTS/check-tools-and-env.sh

The survey contract has 2 participants:
* `survey_participant`: You, the person taking the survey
* `custodian`: The party that owns the survey reward token 

### Setup Survey Participant Keys

In this section we will use the `create-test-wallet.sh` script to create the `survey_participant` keys. Make sure to have a local faucet wallet with funds as described in the [01-setup-keys.ipynb](../../setup/01-setup-keys.ipynb) notebook.



In [ ]:
source $SCRIPTS/create-test-wallet.sh survey_participant
$SCRIPTS/fund-wallets.sh $SURVEY_PARTICIPANT_ADDR

You can use the recovery phrase to restore your account on any [wallet](https://builtoncardano.com/ecosystem/wallets):

You can also use cardano scan to see the transactions and balances of your account:

In [ ]:
$SCRIPTS/cardano-scan-address.sh $SURVEY_PARTICIPANT_ADDR

### Setup the custodian party
The custodian party owns the survey reward token. The address is:

In [ ]:
export CUSTODIAN_ADDR=addr_test1vpcucug827nlrmsv7n66hwdfpemwqtv8nxnjc4azacuu80csxt40z

## Design the contract in the Playground

The onchain survey will consist of 5 questions, the first 4 are statements that you need to agree or disagree and the 5th question is a free text.

1. I’d like to use/recommend Marlowe for future dApps
    * 1 - Higly disagree 
    * 2 - Disagree
    * 3 - Neutral
    * 4 - Agree
    * 5 - Highly agree
2. The survey contract was easy to design
    * 1 - Higly disagree 
    * 2 - Disagree
    * 3 - Neutral
    * 4 - Agree
    * 5 - Highly agree
3. The Marlowe Runtime was easy to work with
    * 1 - Higly disagree 
    * 2 - Disagree
    * 3 - Neutral
    * 4 - Agree
    * 5 - Highly agree
4. I am an experienced web3 developer
    * 1 - Higly disagree 
    * 2 - Disagree
    * 3 - Neutral
    * 4 - Agree
    * 5 - Highly agree
5. Please share any comments (240 char max)

To design the contract we'll use the Blockly editor in the [Marlowe Playground](https://play.marlowe.iohk.io/). The questions will be modeled using `Choice Action`. Each question will be answered by the `Address $SURVEY_PARTICIPANT_ADDR` participant before a parameterized `answer timeout`. If the contract timeouts we close it.

Create the 5 questions blocks with the following parameters:

| Choice name | Bounds    |
|-------------|-----------|
| answer1     | [1 - 5]   |
| answer2     | [1 - 5]   |
| answer3     | [1 - 5]   |
| answer4     | [1 - 5]   |
| answer5     | [1 - 1099511627775]   |

![Survey question block](../../images/survey_question_block.png)

After the questions, the `$CUSTODIAN_ADDR` participant will deposit 1 `Token "6fcbab5bb175b420cd060edb63af74c5b3d4687163f282ddc5377dd3" "SurveyReward"` directly into the survey participant account before a parameterized `reward timeout` and then close the contract. After the contract is closed, the payment of the reward token will be made automatically.

![Survey reward block](../../images/survey_reward_block.png)

After you have created the contract, send it to simulation and check that there is only one path to receive the SurveyReward, which is to answer the 5 questions and for the custodian to make the deposit. 

Set the `reward timeout` bigger than the `answer timeout` way far in the future, download the contract as `survey.json` and upload it to [the notebook folder](.).

![Survey contract parameters](../../images/survey_contract_parameters.png)




Note that we use fixed addresses (so the contract participation cannot be traded) but instead of using the actual addresses we use environment variables. This is possible because the playground does not check the validity of the addresses and inside this notebook we can make the releveant interpolation.

In [ ]:
cat survey.json | \
    sed "s/\$CUSTODIAN_ADDR/$CUSTODIAN_ADDR/g" | \
    sed "s/\$SURVEY_PARTICIPANT_ADDR/$SURVEY_PARTICIPANT_ADDR/g" > survey-with-addr.json

## Transaction 1. Create the Contract

On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [ ]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

We'll add an event name in the tag metadata so that the custodian can find the contract easier

In [ ]:
export EVENT_NAME=CryptoPall2023

A `HTTP` `POST` request to Marlowe Runtime\'s `/contracts` endpoint will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose and the event tag.

We start by creating the json payload as `request-1.json`

In [ ]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat survey-with-addr.json`
roles: null
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: 
  SurveyContract: $EVENT_NAME
EOI
cat request-1.json | jq

Next we post the request and view the response.

In [ ]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SURVEY_PARTICIPANT_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

From the response object we need to get the contract id

In [ ]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

the base link for further restful commands:

In [ ]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL: $CONTRACT_URL"

check that there aren't any safety errors

In [ ]:
jq -r '.resource.safetyErrors' response-1.json

extract the unsigned transaction (serialized as CBOR in text-envelope format)

In [ ]:
jq '.resource.txBody' response-1.json > tx-1.unsigned
du -hs tx-1.unsigned

Then sign and submit the transaction:


In [ ]:
echo "submiting transaction..."
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$SURVEY_PARTICIPANT_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"


We can view the transaction on the Cardano and Marlowe explorer and see that the contract has been created. 
> NOTE: It might take some time for the transaction to be visible in an explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_1
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

## Transaction 2. First answer

Select the option that best describes this sentence:

> **Question 1**: I’d like to use/recommend Marlowe for future dApps
>   * 1 - Higly disagree 
>   * 2 - Disagree
>   * 3 - Neutral
>   * 4 - Agree
>   * 5 - Highly agree

We'll answer the first questiong with the help of Marlowe Runtime\'s 

The `marlowe-cli input choose` tool conveniently formats the correct JSON for a choice.

In [ ]:
ANSWER1=FILL_NUMBER_1-TO-5

We'll use the `marlowe-cli input choose` command and the **HTTP** `POST` `/contract/{contractId}/transactions` endpoint to answer.

In [ ]:
marlowe-cli input choose \
  --choice-name answer1 \
  --choice-party $SURVEY_PARTICIPANT_ADDR \
  --choice-number "$ANSWER1" \
  --out-file answer-1-input.json

yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat answer-1-input.json)]
metadata: {}
tags: {}
EOI

cat request-2.json | jq

Next we post the request and store the response.

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SURVEY_PARTICIPANT_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

echo "submiting transaction..."

TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$SURVEY_PARTICIPANT_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

We can view the transaction on the Cardano and Marlowe explorer.

In [ ]:
echo "Cardano Scan (low level)"
$SCRIPTS/cardano-scan-tx.sh $TX_2
echo
echo "Marlowe Scan (high level)"
$SCRIPTS/marlowe-scan.sh $CONTRACT_ID

## Transaction 3. Answer 2nd to 4th question

Marlowe contracts supports including multiple inputs in a single transaction as long as the execution budget permits it. Here we'll answer questions 2 to 4 in a single transaction.

Select the option that best describes this sentence:

> **Question 2**: The survey contract was easy to design
>   * 1 - Higly disagree 
>   * 2 - Disagree
>   * 3 - Neutral
>   * 4 - Agree
>   * 5 - Highly agree
>
> **Question 3**: The Marlowe Runtime was easy to work with
>   * 1 - Higly disagree 
>   * 2 - Disagree
>   * 3 - Neutral
>   * 4 - Agree
>   * 5 - Highly agree
>
> **Question 4**: I am an experienced web3 developer
>   * 1 - Higly disagree 
>   * 2 - Disagree
>   * 3 - Neutral
>   * 4 - Agree
>   * 5 - Highly agree

In [ ]:
ANSWER2=FILL_NUMBER_1-TO-5
ANSWER3=FILL_NUMBER_1-TO-5
ANSWER4=FILL_NUMBER_1-TO-5

Again we'll use the `marlowe-cli input choose` to select the choices

In [ ]:
marlowe-cli input choose \
  --choice-name answer2 \
  --choice-party $SURVEY_PARTICIPANT_ADDR \
  --choice-number "$ANSWER2" \
  --out-file answer-2-input.json

marlowe-cli input choose \
  --choice-name answer3 \
  --choice-party $SURVEY_PARTICIPANT_ADDR \
  --choice-number "$ANSWER3" \
  --out-file answer-3-input.json

marlowe-cli input choose \
  --choice-name answer4 \
  --choice-party $SURVEY_PARTICIPANT_ADDR \
  --choice-number "$ANSWER4" \
  --out-file answer-4-input.json


and the **HTTP** `POST` `/contract/{contractId}/transactions` endpoint to create the transaction

In [ ]:
yaml2json << EOI > request-3.json
version: v1
inputs: 
 - $(cat answer-2-input.json)
 - $(cat answer-3-input.json)
 - $(cat answer-4-input.json)
metadata: {}
tags: {}
EOI
cat request-3.json | jq

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SURVEY_PARTICIPANT_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

Once again, use `marlowe-cli` to sign and submit the transaction and then wait for confirmation.

In [ ]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

echo "submiting transaction..."

TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$SURVEY_PARTICIPANT_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

## Transaction 4. Free text answer
Marlowe choices are numeric, so we can't use the same mechanism to submit free text. Instead, we will encrypt the answer using a known public key from the custodian (the only one that can decrypt it) and submit a numeric hash of the answer as the Marlowe choice.

The encryption provide us privacy, only allowing the `survey_participant` and the `custodian` to know the `SECRET_ANSWER`. The hash provides us authenticity, allowing the `custodian` to verify that the `survey_participant` has not changed the answer after submitting it.

In [ ]:
# A free form answer with max of 240 characters
export SECRET_ANSWER=$(cat <<'EOI'
Marlowe is cool!!!
EOI
)

The 240 character limitation is a limitation of the encription algorithm being used at the moment. A different algorithm could be used to allow for longer messages. 

In [ ]:
if [ ${#SECRET_ANSWER} -gt 240 ]; then
    echo "SECRET_ANSWER is too long"
fi
echo "SECRET_ANSWER has ${#SECRET_ANSWER} chars"

To encrypt the `SECRET_ANSWER` we'll use the custodian public key with openssl

In [ ]:
cat << EOI > custodian.pem
-----BEGIN PUBLIC KEY-----
MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAyTHkM+HNQ7M+T2bEW9cq
kdjQ49AYY/M4+q1LawK0EheeFX3OxW97muysn/kWuIrF4gcCQVB5Fuiys43LbHmv
0hzrJNmure1C2T0yFW56MO4kv5n/LHYYZttyHdTbQS0og/XaO5VhtvdzuIFnCaf+
64zIWYnzLs/zSwCbX6iORXxdMF7C6h8QOtNZuLVXwgBJsvZfNhN4kyfYLA+EyJJ6
2hUdpHDA2/qQ1ydk2wYHy9oj9xMQ6hUz2DvmsqqONtk15uNWy2zGxlBTQeImuZ72
0HvUBeh9mQcFjTHdPysL96Ty0XZINQ29izJqOi13JQV0eY+V/9RZn81lFAtGJhAC
XQIDAQAB
-----END PUBLIC KEY-----

EOI

export SECRET_ANSWER_ENC=$(openssl pkeyutl \
                            -encrypt \
                            -inkey custodian.pem \
                            -pubin -in <(echo -n $SECRET_ANSWER) \
                            -out /dev/stdout | base64 | tr -d '\n'\
                          )

echo "Encrypted secret answer:"
echo
echo $SECRET_ANSWER_ENC

We can post the encrypted result in a public url using a service like:
- [pastebin](https://pastebin.com/)
- [ipfs](https://ipfs.io/)
- [S3 bucket](https://aws.amazon.com/s3/)
- [gist](https://gist.github.com/) (warning: the owner of the gist is public)


In [ ]:
export SECRET_ANSWER_ENC_URL=https://pastebin.com/vstTD1Sb

We cannot prevent the participant or a malicious actor to modify the result (anybody can encrypt a message using the `custodian` public key), but we can use `openssl` to hash the `SECRET_ANSWER` into a 32 Byte hex string. 

The result would be a really big decimal number (`2^128 - 1`) so we can use the last 10 Bytes as a sub-hash. This increases the chances of hash collision, as the amount of possibilites is reduced to `2^40 - 1 = 1.099.511.627.775`, which is acceptable for this example.

In [ ]:
export SECRET_ANSWER_HASH=$(echo -n "$SECRET_ANSWER" | openssl dgst -sha256 | cut -d' ' -f2)
export SECRET_ANSWER_HASH_LAST_DIGITS=$(echo -n "$SECRET_ANSWER_HASH" | tail -c 10)
export SECRET_ANSWER_HASH_LAST_DIGITS_DECIMAL=$(printf "%d" "0x$SECRET_ANSWER_HASH_LAST_DIGITS")

echo "* SECRET_ANSWER_HASH = $SECRET_ANSWER_HASH"
echo "* SECRET_ANSWER_HASH_LAST_DIGITS = $SECRET_ANSWER_HASH_LAST_DIGITS"
echo "* SECRET_ANSWER_HASH_LAST_DIGITS_DECIMAL = $SECRET_ANSWER_HASH_LAST_DIGITS_DECIMAL"

echo "Max choice: $(printf "%d" "0xffffffffff")"

Once again we use the `marlowe-cli input choose` and the **HTTP** `POST` `/contract/{contractId}/transactions` endpoint to create the transaction

In [ ]:

marlowe-cli input choose \
  --choice-name answer5 \
  --choice-party $SURVEY_PARTICIPANT_ADDR \
  --choice-number "$SECRET_ANSWER_HASH_LAST_DIGITS_DECIMAL" \
  --out-file answer-5-input.json


yaml2json << EOI > request-4.json
version: v1
inputs: [$(cat answer-5-input.json)]
metadata: {}
tags:
  encryptedAnswer: $SECRET_ANSWER_ENC_URL
EOI
cat request-4.json | jq

In [ ]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SURVEY_PARTICIPANT_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

In [ ]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

echo "submiting transaction..."

TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$SURVEY_PARTICIPANT_SKEY" \
  --timeout 600s \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

## Wait for reward

Congratulations! You have successfully completed the survey. The `custodian` will need to deposit the reward to finish the contract. For the moment this process is not automated, so you can contact the [#ask-marlowe](https://discord.com/channels/826816523368005654/936295815926927390) channel on discord using your address and contract id

In [ ]:
echo "Contract id = $CONTRACT_ID"
echo "Survey participant address = $SURVEY_PARTICIPANT_ADDR"    

You can check in cardano scan to see if you received the reward

In [ ]:
$SCRIPTS/cardano-scan-address.sh $SURVEY_PARTICIPANT_ADDR

## Return the remaining ADA to the local faucet
Once the contract is closed and you claimed your reward, you can use the following command to send the remaining ADA back to the local faucet.


In [ ]:
$SCRIPTS/return-assets-to-faucet.sh survey_participant